# Name : Rishabh Rajput
# Course : MCA
# Roll no: 2301560003

# Question no 3
# String Matching

In [2]:
pip install fuzzywuzzy

In [3]:
from fuzzywuzzy import fuzz, process

C:\Users\admin\Desktop\New folder\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
berlin = ['Berlin, Germany', 
          'Berlin, Deutschland', 
          'Berlin', 
          'Berlin, DE']

In [5]:
#Try matching the first and second strings: 'Berlin, Germany' and 'Berlin, Deutschland'
fuzz.partial_ratio(berlin[0], berlin[1])


60

In [6]:
fuzz.ratio?

In [7]:
fuzz.ratio(berlin[0], berlin[1])

65

In [8]:
fuzz.token_set_ratio(berlin[0], berlin[1])

62

In [9]:
fuzz.token_sort_ratio(berlin[0], berlin[1])

62

In [10]:
#Try matching the second and third strings: 'Berlin, Deutschland' and 'Berlin'
fuzz.partial_ratio(berlin[1], berlin[2])

100

In [11]:
fuzz.ratio(berlin[1], berlin[2])

50

In [12]:
fuzz.token_sort_ratio(berlin[1], berlin[2])

50

In [13]:
#What do you think will score lowest and highest for the final two elements:
#'Berlin'
#'Berlin, DE'
fuzz.token_set_ratio(berlin[2], berlin[3])

100

In [14]:
#Extracting a guess out of a list
choices = ['Germany', 'Deutschland', 'France', 
           'United Kingdom', 'Great Britain', 
           'United States']

In [15]:
process.extract('DE', choices, limit=2)

[('Deutschland', 90), ('United States', 57)]

In [16]:
process.extract('UK', choices)

[('Deutschland', 45),
 ('United Kingdom', 45),
 ('United States', 45),
 ('Germany', 0),
 ('France', 0)]

In [17]:
process.extract('frankreich', choices)

[('France', 62),
 ('Great Britain', 41),
 ('Germany', 35),
 ('United Kingdom', 25),
 ('United States', 25)]

In [18]:
#Will this properly extract?
process.extract('U.S.', choices)

[('United States', 86),
 ('Deutschland', 60),
 ('United Kingdom', 57),
 ('Great Britain', 30),
 ('Germany', 0)]

# Managing Nulls with Pandas 

In [19]:
import pandas as pd
from numpy import random
df = pd.read_csv('iot_example_with_nulls.csv')

In [22]:
df.head()

,timestamp,username,temperature,heartrate,build,latest,note
0,2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
1,2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2,2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
3,2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
4,2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,NaN,NaN


In [23]:
df.dtypes


timestamp       object
username        object
temperature    float64
heartrate        int64
build           object
latest         float64
note            object
dtype: object

In [24]:
df.note.value_counts()

note
wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: count, dtype: int64

In [25]:
#Let's remove all null values (including the note: n/a)
df = pd.read_csv('iot_example_with_nulls.csv', 
                 na_values=['n/a'])

In [26]:
#Test to see if we can use dropna
df.shape

(146397, 7)

In [27]:
df.dropna().shape

(46116, 7)

In [28]:
df.dropna(how='all', axis=1).shape

(146397, 7)

In [30]:
#Test to see if we can drop columns
my_columns = list(df.columns)
my_columns

['timestamp',
 'username',
 'temperature',
 'heartrate',
 'build',
 'latest',
 'note']

In [31]:
list(df.dropna(thresh=int(df.shape[0] * .9), axis=1).columns)

['timestamp', 'username', 'heartrate']

In [32]:
#I want to find all columns that have missing data
missing_info = list(df.columns[df.isnull().any()])
missing_info

['temperature', 'build', 'latest', 'note']

In [36]:
for col in missing_info:
    num_missing = df[df[col].isnull() == True].shape[0]
    print('number missing for column {}: {}'.format(col, 
                                                    num_missing))
print("\n")
print("#######################################################\n")
for col in missing_info:
    percent_missing = df[df[col].isnull() == True].shape[0] / df.shape[0]
    print('percent missing for column {}: {}'.format(
        col, percent_missing))

number missing for column temperature: 32357
number missing for column build: 32350
number missing for column latest: 32298
number missing for column note: 48704


#######################################################

percent missing for column temperature: 0.22102228870810195
percent missing for column build: 0.22097447352063226
percent missing for column latest: 0.22061927498514314
percent missing for column note: 0.332684412931959


In [43]:
#Can I easily substitute majority values in for missing data?
df.note.value_counts()
df.build.value_counts().head()
df.latest.value_counts()
df.latest = df.latest.fillna(0)

In [52]:
#Have not yet addressed temperature missing values... Let's find a way to fill
df.username.value_counts().head()
df.head()
df.temperature = df.groupby('username').temperature.fillna(
    method='backfill', limit=3)
df.head()

,username,temperature,heartrate,build,latest,note
timestamp,,,,,,
2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,0.0,NaN


# Scikit Learn Preprocessing

In [61]:
from sklearn import preprocessing
from datetime import datetime
hvac = pd.read_csv('HVAC_with_nulls.csv')

In [62]:
#Checking Data Quality
hvac.dtypes

Date           object
Time           object
TargetTemp    float64
ActualTemp      int64
System          int64
SystemAge     float64
BuildingID      int64
10            float64
dtype: object

In [63]:
hvac.shape

(8000, 8)

In [64]:
hvac.head()

,Date,Time,TargetTemp,ActualTemp,System,SystemAge,BuildingID,10
0,6/1/13,0:00:01,66.0,58,13,20.0,4,NaN
1,6/2/13,1:00:01,NaN,68,3,20.0,17,NaN
2,6/3/13,2:00:01,70.0,73,17,20.0,18,NaN
3,6/4/13,3:00:01,67.0,63,2,NaN,15,NaN
4,6/5/13,4:00:01,68.0,74,16,9.0,3,NaN


In [67]:
#Scale temperature values
hvac['ScaledTemp'] = preprocessing.scale(hvac['ActualTemp'])
hvac['ScaledTemp'].head()

0   -1.293272
1    0.048732
2    0.719733
3   -0.622270
4    0.853934
Name: ScaledTemp, dtype: float64

In [68]:
#Scale using a min and max scaler
min_max_scaler = preprocessing.MinMaxScaler()
temp_minmax = min_max_scaler.fit_transform(hvac[['ActualTemp']])
temp_minmax

array([[0.12],
       [0.52],
       [0.72],
       ...,
       [0.56],
       [0.32],
       [0.44]])

# Case Study: Preparing Lobste.rs Stories for Machine Learning

In [69]:
import requests
from fuzzywuzzy import fuzz
from collections import Counter
from sklearn import preprocessing

In [70]:
#If you'd rather read from the API to get the latest, uncomment the details (and add comment to the final line)
#resp = requests.get('https://lobste.rs/hottest.json')
#stories = pd.read_json(resp.content)
#stories = stories.set_index('short_id')

stories = pd.read_json('all_lobsters.json')

In [71]:
stories.head()

,comment_count,comments_url,created_at,description,downvotes,last_updated,score,short_id_url,submitter_user,tags,title,upvotes,url
09zw7r,0,https://lobste.rs/s/09zw7r/edited_truth,2017-08-08 20:11:09,,0,2017-08-09T11:03:57.014269,3,https://lobste.rs/s/09zw7r,{'avatar_url': 'https://lobste.rs/avatars/trn-...,"[crypto, pdf]",The Edited Truth,3,https://eprint.iacr.org/2017/714.pdf
0bdne7,17,https://lobste.rs/s/0bdne7/rise_social_media_v...,2017-08-08 21:12:38,,9,2017-08-09T11:03:57.014269,-1,https://lobste.rs/s/0bdne7,{'avatar_url': 'https://lobste.rs/avatars/nkhu...,"[law, privacy]",The Rise of The Social Media Vigilante,8,https://medium.com/@nkhumphreys_89452/the-rise...
1bhbod,11,https://lobste.rs/s/1bhbod/tcl_misunderstood_a...,2017-04-30 20:28:52,<p>Did any language end up taking that “highly...,0,2017-05-01T06:29:11.725518,17,https://lobste.rs/s/1bhbod,"{'is_moderator': False, 'is_admin': False, 'us...",[programming],Tcl the misunderstood - antirez,17,http://antirez.com/articoli/tclmisunderstood.html
1xkje1,0,https://lobste.rs/s/1xkje1/interview_4_jonatha...,2017-05-01 02:31:35,<p>Rust’s own Jonathan Turner on his backgroun...,0,2017-05-01T06:29:11.725518,1,https://lobste.rs/s/1xkje1,"{'is_moderator': False, 'is_admin': False, 'us...","[audio, javascript, rust]",🎤🎙 Interview 4 – Jonathan Turner: Part 1/3,1,http://www.newrustacean.com/show_notes/intervi...
2dasvh,19,https://lobste.rs/s/2dasvh/return_hipster_pda,2017-08-08 14:25:29,,0,2017-08-09T11:03:56.287654,20,https://lobste.rs/s/2dasvh,{'created_at': '2017-01-19T14:56:50.000-06:00'...,[practices],The Return of the Hipster PDA,20,http://www.agilesysadmin.net/return-of-the-hip...


In [73]:
stories.dtypes

comment_count              int64
comments_url              object
created_at        datetime64[ns]
description               object
downvotes                  int64
last_updated              object
score                      int64
short_id_url              object
submitter_user            object
tags                      object
title                     object
upvotes                    int64
url                       object
dtype: object

In [74]:
#Let's take a look at the submitter_user field, as it appears like a dict
stories.submitter_user.iloc[3]

{'is_moderator': False,
 'is_admin': False,
 'username': 'chriskrycho',
 'karma': 27,
 'avatar_url': 'https://secure.gravatar.com/avatar/c096ed07142659408dc6651f8320acd3?r=pg&d=identicon&s=100',
 'created_at': '2016-08-15T09:33:28.000-05:00',
 'about': "I'm a husband and father; a theologian, composer, poet, and essayist; a front end developer at [Olo](http://www.olo.com); a [Rust](https://www.rust-lang.org/en-US/) enthusiast host; and the host of the [Winning Slowly](http://www.winningslowly.org), [New Rustacean](http://www.newrustacean.com/), [Sap.py](http://www.sap-py.com), and [Run With Me](http://runwith.chriskrycho.com/) podcasts."}

In [75]:
user_df = stories['submitter_user'].apply(pd.Series)

In [76]:
user_df.head()

,avatar_url,created_at,is_admin,username,karma,is_moderator,about,github_username
09zw7r,https://lobste.rs/avatars/trn-100.png,2017-01-19T14:56:50.000-06:00,False,trn,429,False,,NaN
0bdne7,https://lobste.rs/avatars/nkhumphreys-100.png,2014-07-02T06:36:39.000-05:00,False,nkhumphreys,-1,False,Web developer and previously embedded C developer,NaN
1bhbod,https://secure.gravatar.com/avatar/85002353297...,2016-11-30T10:14:24.000-06:00,False,yumaikas,578,False,I blog infrequently at https://junglecoder.com...,NaN
1xkje1,https://secure.gravatar.com/avatar/c096ed07142...,2016-08-15T09:33:28.000-05:00,False,chriskrycho,27,False,"I'm a husband and father; a theologian, compos...",NaN
2dasvh,https://lobste.rs/avatars/trn-100.png,2017-01-19T14:56:50.000-06:00,False,trn,429,False,,NaN


In [77]:
#Can we combine the user data without potential column overlap?
set(user_df.columns).intersection(stories.columns)

{'created_at'}

In [78]:
user_df = user_df.rename(columns={'created_at': 
                                  'user_created_at'})

In [79]:
stories = pd.concat([stories.drop(['submitter_user'], axis=1), 
                     user_df], axis=1)

In [80]:
stories.head()

,comment_count,comments_url,created_at,description,downvotes,last_updated,score,short_id_url,tags,title,upvotes,url,avatar_url,user_created_at,is_admin,username,karma,is_moderator,about,github_username
09zw7r,0,https://lobste.rs/s/09zw7r/edited_truth,2017-08-08 20:11:09,,0,2017-08-09T11:03:57.014269,3,https://lobste.rs/s/09zw7r,"[crypto, pdf]",The Edited Truth,3,https://eprint.iacr.org/2017/714.pdf,https://lobste.rs/avatars/trn-100.png,2017-01-19T14:56:50.000-06:00,False,trn,429,False,,NaN
0bdne7,17,https://lobste.rs/s/0bdne7/rise_social_media_v...,2017-08-08 21:12:38,,9,2017-08-09T11:03:57.014269,-1,https://lobste.rs/s/0bdne7,"[law, privacy]",The Rise of The Social Media Vigilante,8,https://medium.com/@nkhumphreys_89452/the-rise...,https://lobste.rs/avatars/nkhumphreys-100.png,2014-07-02T06:36:39.000-05:00,False,nkhumphreys,-1,False,Web developer and previously embedded C developer,NaN
1bhbod,11,https://lobste.rs/s/1bhbod/tcl_misunderstood_a...,2017-04-30 20:28:52,<p>Did any language end up taking that “highly...,0,2017-05-01T06:29:11.725518,17,https://lobste.rs/s/1bhbod,[programming],Tcl the misunderstood - antirez,17,http://antirez.com/articoli/tclmisunderstood.html,https://secure.gravatar.com/avatar/85002353297...,2016-11-30T10:14:24.000-06:00,False,yumaikas,578,False,I blog infrequently at https://junglecoder.com...,NaN
1xkje1,0,https://lobste.rs/s/1xkje1/interview_4_jonatha...,2017-05-01 02:31:35,<p>Rust’s own Jonathan Turner on his backgroun...,0,2017-05-01T06:29:11.725518,1,https://lobste.rs/s/1xkje1,"[audio, javascript, rust]",🎤🎙 Interview 4 – Jonathan Turner: Part 1/3,1,http://www.newrustacean.com/show_notes/intervi...,https://secure.gravatar.com/avatar/c096ed07142...,2016-08-15T09:33:28.000-05:00,False,chriskrycho,27,False,"I'm a husband and father; a theologian, compos...",NaN
2dasvh,19,https://lobste.rs/s/2dasvh/return_hipster_pda,2017-08-08 14:25:29,,0,2017-08-09T11:03:56.287654,20,https://lobste.rs/s/2dasvh,[practices],The Return of the Hipster PDA,20,http://www.agilesysadmin.net/return-of-the-hip...,https://lobste.rs/avatars/trn-100.png,2017-01-19T14:56:50.000-06:00,False,trn,429,False,,NaN


In [81]:
#Let's check for nulls
stories.shape

(74, 20)

In [82]:
stories.dropna().shape

(8, 20)

In [83]:
stories.dropna(thresh=10, axis=1).shape

(74, 19)

In [84]:
#Let's make the tags easier to use by having them as features in the columns.
tag_df = stories.tags.apply(pd.Series)

In [85]:
tag_df.head()

,0,1,2,3,4
09zw7r,crypto,pdf,NaN,NaN,NaN
0bdne7,law,privacy,NaN,NaN,NaN
1bhbod,programming,NaN,NaN,NaN,NaN
1xkje1,audio,javascript,rust,NaN,NaN
2dasvh,practices,NaN,NaN,NaN,NaN


In [86]:
pd.unique(tag_df.values.ravel())

array(['crypto', 'pdf', nan, 'law', 'privacy', 'programming', 'audio',
       'javascript', 'rust', 'practices', 'ruby', 'devops', 'web',
       'hardware', 'science', 'reversing', 'security', 'openbsd',
       'windows', 'design', 'compilers', 'haskell', 'c++', 'assembly',
       'games', 'math', 'release', 'event', 'netbsd', 'unix', 'c',
       'linux', 'testing', 'lua', 'job', 'video', 'philosophy', 'android',
       'networking', 'erlang', 'emacs', 'historical', 'browsers',
       'person', 'culture', 'java', 'go', 'book', 'css', 'debugging',
       'education', 'art', 'compsci', 'databases'], dtype=object)

In [87]:
set(tag_df.values.ravel())

{'android',
 'art',
 'assembly',
 'audio',
 'book',
 'browsers',
 'c',
 'c++',
 'compilers',
 'compsci',
 'crypto',
 'css',
 'culture',
 'databases',
 'debugging',
 'design',
 'devops',
 'education',
 'emacs',
 'erlang',
 'event',
 'games',
 'go',
 'hardware',
 'haskell',
 'historical',
 'java',
 'javascript',
 'job',
 'law',
 'linux',
 'lua',
 'math',
 nan,
 'netbsd',
 'networking',
 'openbsd',
 'pdf',
 'person',
 'philosophy',
 'practices',
 'privacy',
 'programming',
 'release',
 'reversing',
 'ruby',
 'rust',
 'science',
 'security',
 'testing',
 'unix',
 'video',
 'web',
 'windows'}

In [88]:
len(pd.unique(tag_df.values.ravel()))


54

In [89]:
# most common tags

Counter(tag_df.values.ravel()).most_common(5)

[(nan, 231),
 ('programming', 13),
 ('hardware', 10),
 ('security', 10),
 ('practices', 8)]

In [91]:
#Let's create a dummy df with our tags
tag_df = pd.get_dummies(
    tag_df.apply(pd.Series).stack()).sum()

In [92]:
tag_df.head()

android     1
art         1
assembly    3
audio       1
book        2
dtype: int64

In [93]:
#Now we can add it back to our stories DataFrame
stories = pd.concat([stories.drop('tags', axis=1), 
                     tag_df], axis=1)

In [94]:
stories.head()

,comment_count,comments_url,created_at,description,downvotes,last_updated,score,short_id_url,title,upvotes,url,avatar_url,user_created_at,is_admin,username,karma,is_moderator,about,github_username,0
09zw7r,0.0,https://lobste.rs/s/09zw7r/edited_truth,2017-08-08 20:11:09,,0.0,2017-08-09T11:03:57.014269,3.0,https://lobste.rs/s/09zw7r,The Edited Truth,3.0,https://eprint.iacr.org/2017/714.pdf,https://lobste.rs/avatars/trn-100.png,2017-01-19T14:56:50.000-06:00,False,trn,429.0,False,,NaN,NaN
0bdne7,17.0,https://lobste.rs/s/0bdne7/rise_social_media_v...,2017-08-08 21:12:38,,9.0,2017-08-09T11:03:57.014269,-1.0,https://lobste.rs/s/0bdne7,The Rise of The Social Media Vigilante,8.0,https://medium.com/@nkhumphreys_89452/the-rise...,https://lobste.rs/avatars/nkhumphreys-100.png,2014-07-02T06:36:39.000-05:00,False,nkhumphreys,-1.0,False,Web developer and previously embedded C developer,NaN,NaN
1bhbod,11.0,https://lobste.rs/s/1bhbod/tcl_misunderstood_a...,2017-04-30 20:28:52,<p>Did any language end up taking that “highly...,0.0,2017-05-01T06:29:11.725518,17.0,https://lobste.rs/s/1bhbod,Tcl the misunderstood - antirez,17.0,http://antirez.com/articoli/tclmisunderstood.html,https://secure.gravatar.com/avatar/85002353297...,2016-11-30T10:14:24.000-06:00,False,yumaikas,578.0,False,I blog infrequently at https://junglecoder.com...,NaN,NaN
1xkje1,0.0,https://lobste.rs/s/1xkje1/interview_4_jonatha...,2017-05-01 02:31:35,<p>Rust’s own Jonathan Turner on his backgroun...,0.0,2017-05-01T06:29:11.725518,1.0,https://lobste.rs/s/1xkje1,🎤🎙 Interview 4 – Jonathan Turner: Part 1/3,1.0,http://www.newrustacean.com/show_notes/intervi...,https://secure.gravatar.com/avatar/c096ed07142...,2016-08-15T09:33:28.000-05:00,False,chriskrycho,27.0,False,"I'm a husband and father; a theologian, compos...",NaN,NaN
2dasvh,19.0,https://lobste.rs/s/2dasvh/return_hipster_pda,2017-08-08 14:25:29,,0.0,2017-08-09T11:03:56.287654,20.0,https://lobste.rs/s/2dasvh,The Return of the Hipster PDA,20.0,http://www.agilesysadmin.net/return-of-the-hip...,https://lobste.rs/avatars/trn-100.png,2017-01-19T14:56:50.000-06:00,False,trn,429.0,False,,NaN,NaN


In [95]:
#Another potentially useful feature is the post times...
stories['created_hour'] = stories.created_at.map(
    lambda x: x.hour)


In [96]:
stories['created_dow'] = stories.created_at.map(
    lambda x: x.weekday())

In [97]:
#Let's analyze some of the correlations in our features so far...
stories[['created_hour', 'score']].corr()


,created_hour,score
created_hour,1.000000,0.253917
score,0.253917,1.000000


In [98]:
stories[['created_dow', 'score']].corr()


,created_dow,score
created_dow,1.000000,-0.113918
score,-0.113918,1.000000


In [99]:
stories[['karma', 'score']].corr()


,karma,score
karma,1.000000,-0.061921
score,-0.061921,1.000000


In [100]:
stories[['comment_count', 'score']].corr()


,comment_count,score
comment_count,1.000000,0.637632
score,0.637632,1.000000


In [108]:
scaler = preprocessing.MinMaxScaler()
scaled_score = scaler.fit_transform(stories[['score']])
scaled_score[:5]
stories['scaled_score'] = scaled_score[:,0]
stories.head()

,comment_count,comments_url,created_at,description,downvotes,last_updated,score,short_id_url,title,upvotes,...,is_admin,username,karma,is_moderator,about,github_username,0,created_hour,created_dow,scaled_score
09zw7r,0.0,https://lobste.rs/s/09zw7r/edited_truth,2017-08-08 20:11:09,,0.0,2017-08-09T11:03:57.014269,3.0,https://lobste.rs/s/09zw7r,The Edited Truth,3.0,...,False,trn,429.0,False,,NaN,NaN,20.0,1.0,0.072727
0bdne7,17.0,https://lobste.rs/s/0bdne7/rise_social_media_v...,2017-08-08 21:12:38,,9.0,2017-08-09T11:03:57.014269,-1.0,https://lobste.rs/s/0bdne7,The Rise of The Social Media Vigilante,8.0,...,False,nkhumphreys,-1.0,False,Web developer and previously embedded C developer,NaN,NaN,21.0,1.0,0.000000
1bhbod,11.0,https://lobste.rs/s/1bhbod/tcl_misunderstood_a...,2017-04-30 20:28:52,<p>Did any language end up taking that “highly...,0.0,2017-05-01T06:29:11.725518,17.0,https://lobste.rs/s/1bhbod,Tcl the misunderstood - antirez,17.0,...,False,yumaikas,578.0,False,I blog infrequently at https://junglecoder.com...,NaN,NaN,20.0,6.0,0.327273
1xkje1,0.0,https://lobste.rs/s/1xkje1/interview_4_jonatha...,2017-05-01 02:31:35,<p>Rust’s own Jonathan Turner on his backgroun...,0.0,2017-05-01T06:29:11.725518,1.0,https://lobste.rs/s/1xkje1,🎤🎙 Interview 4 – Jonathan Turner: Part 1/3,1.0,...,False,chriskrycho,27.0,False,"I'm a husband and father; a theologian, compos...",NaN,NaN,2.0,0.0,0.036364
2dasvh,19.0,https://lobste.rs/s/2dasvh/return_hipster_pda,2017-08-08 14:25:29,,0.0,2017-08-09T11:03:56.287654,20.0,https://lobste.rs/s/2dasvh,The Return of the Hipster PDA,20.0,...,False,trn,429.0,False,,NaN,NaN,14.0,1.0,0.381818


# Tracking the International Space Station with Dask

In [111]:
import requests
import logging
import sys
from time import sleep
from datetime import datetime
from math import radians
from dask import delayed
from operator import itemgetter

In [112]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [113]:
#First, we need to get lat and long pairs from a list of cities
def get_lat_long(address):
    resp = requests.get(
        'https://eu1.locationiq.org/v1/search.php',
        params={'key': '92e7ba84cf3465', #Please be kind, you can generate your own for more use here - https://locationiq.org :D
                'q': address,
                'format': 'json'}
    )
    if resp.status_code != 200:
        print('There was a problem with your request!')
        print(resp.content)
        return
    data = resp.json()[0]
    return {
        'name': data.get('display_name'),
        'lat': float(data.get('lat')),
        'long': float(data.get('lon')),
    }

In [114]:
get_lat_long('Berlin, Germany')

There was a problem with your request!
b'{"error":"Invalid key"}'


In [115]:
locations = []
for city in ['Seattle, Washington', 'Miami, Florida', 
             'Berlin, Germany', 'Singapore', 
             'Wellington, New Zealand',
             'Beirut, Lebanon', 'Beijing, China', 'Nairobi, Kenya',
             'Cape Town, South Africa', 'Buenos Aires, Argentina']:
    locations.append(get_lat_long(city))
    sleep(2)

There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'
There was a problem with your request!
b'{"error":"Invalid key"}'


In [116]:
locations

[None, None, None, None, None, None, None, None, None, None]

In [117]:
#Now we can define the functions we will use to get the ISS data and compare location and next pass times amongst cities
def get_spaceship_location():
    resp = requests.get('http://api.open-notify.org/iss-now.json')
    location = resp.json()['iss_position']
    return {'lat': float(location.get('latitude')),
            'long': float(location.get('longitude'))}

In [118]:
def great_circle_dist(lon1, lat1, lon2, lat2):
    "Found on SO: http://stackoverflow.com/a/41858332/380442"
    dist = DistanceMetric.get_metric('haversine')
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    X = [[lat1, lon1], [lat2, lon2]]
    kms = 6367
    return (kms * dist.pairwise(X)).max()

In [119]:
def iss_dist_from_loc(issloc, loc):
    distance = great_circle_dist(issloc.get('long'), 
                                 issloc.get('lat'), 
                                 loc.get('long'), loc.get('lat'))
    logging.info('ISS is ~%dkm from %s', int(distance), loc.get('name'))
    return distance

In [120]:
def iss_pass_near_loc(loc):
    resp = requests.get('http://api.open-notify.org/iss-pass.json',
                        params={'lat': loc.get('lat'), 
                                'lon': loc.get('long')})
    data = resp.json().get('response')[0]
    td = datetime.fromtimestamp(data.get('risetime')) - datetime.now()
    m, s = divmod(int(td.total_seconds()), 60)
    h, m = divmod(m, 60)
    logging.info('ISS will pass near %s in %02d:%02d:%02d',loc.get('name'), h, m, s)
    return td.total_seconds()

In [127]:
#Let's create a delayed pipeline
output = []

for loc in locations:
    issloc = delayed(get_spaceship_location)()
    dist = delayed(iss_dist_from_loc)(issloc, loc)

closest = delayed(lambda x: sorted(x, 
                                   key=itemgetter(1))[0])(output)

In [128]:
closest

Delayed('lambda-18936bf9-bac1-42fb-8393-3c81780ee660')

In [130]:
pip install graphviz

     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------- --------------------- 20.5/47.0 kB 330.3 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/47.0 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 47.0/47.0 kB 391.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [133]:
#Let's see our DAG!
closest.visualize()
closest.head()

Delayed('head-af1a3019-966c-415a-ab1f-e5b61c0966bf')